# Installing Dependencies/Libraries

In [ ]:
#pip install streamlit

In [ ]:
#pip install transformers

In [ ]:
#pip install torch torchvision

In [ ]:
#pip install pypdf2

In [ ]:
#nltk.download('maxent_ne_chunker')

In [ ]:
#nltk.download('words')

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
import json
from difflib import get_close_matches
import json
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW, AutoTokenizer, AutoModelForQuestionAnswering
import torch.nn.functional as F
#import nltk
#from nltk.tokenize import word_tokenize, sent_tokenize
import torch
from torch.utils.data import DataLoader

# Importing Dataset(s)

Book1 => Rules of Play - Game Design Fundamentals

Book2 => Game Design Workshop A Playcentric Approach to Creating Innovative Games

Book3 => The Art of Game Design A Book of Lenses

In [ ]:
# Use the PdfReader class from the pdfplumber module to read the PDF file 'Book1.pdf'
pdf1 = PdfReader('Book1.pdf')

# pdf2 = PdfReader('Book2.pdf')
# pdf3 = PdfReader('Book3.pdf')

Reading the Dataset

In [ ]:
pdf1_data = []

# Iterate over each page in the PDF
for page in pdf1.pages:
    
    # Extract the text from the page and append it to the list
    pdf1_data.append(page.extract_text())

# Test to see if the data is being read correctly
#print(pdf1_data[22])

In [ ]:
# pdf2_data = []
# for page in pdf2.pages:
#     pdf2_data.append(page.extract_text())

In [ ]:
# pdf3_data = []
# for page in pdf3.pages:
#     pdf3_data.append(page.extract_text())

# Data Exploration

In [ ]:
def search_word(word, pdf_data):

    # Initialize a counter for the word
    count = 0

    # Iterate over each page in the PDF data
    for page in pdf_data:

        # Increment the counter by the number of occurrences of the word in the page
        count += page.lower().count(word)

    # Return the total count of the word in the PDF data
    return count

# Test the function with the word 'game' and the first PDF data
#print(search_word('game', pdf1_data))

# Data Processing

Define a function to answer a question based on a given text

In [ ]:
# Define a function to answer a question based on a given text
def answer_question(question, text):
    # Specify the model name
    model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'  # replace with the model of your choice
    
    # Load the tokenizer and model from the Hugging Face Model Hub
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)

    # Tokenize the question and text, and prepare the inputs for the model
    inputs = tokenizer(question, text, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Feed the inputs to the model
    outputs = model(input_ids, attention_mask=attention_mask)
    
    # Get the start and end scores from the model outputs
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Find the start and end indices of the answer in the input_ids
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    # Convert the answer from token ids to string
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index+1]))
    
    # Return the answer
    return answer

# Test the function with a question and a text
#print(answer_question('Is this book for game developers?', pdf1_data[22]))

# Knowledge Base Function

Loading the knowledge_base.json file

In [ ]:
def load_knowledge_base(file_path: str) -> dict:

    # Open the file in read mode
    with open(file_path, 'r') as file:

        # Use the json.load function to load the data from the file
        data: dict = json.load(file)
        
    # Return the loaded data
    return data

Writing to the knowledge_base.json file

In [ ]:
def save_knowledge_base(file_path: str, data: dict):

    # Open the file in write mode
    with open(file_path, 'w') as file:

        # Use the json.dump function to write the data to the file
        # The indent parameter is 2 for pretty printing
        json.dump(data, file, indent=2)

Finding the similarity b/w input and knowledge base to give best response

"cutoff" can be varied to change similarity threshold

In [ ]:
def best_match(user_question: str, questions: list[str]) -> str | None:

    # Use the get_close_matches function from the difflib module to find the closest matches
    matches = get_close_matches(user_question, questions, n=1, cutoff=0.6)
    
    # If no matches are found, return None
    if not matches:
        return None
    
    # Otherwise, return the first (and only) match
    return matches[0]

Picking an answer from the knowledge base

In [ ]:
def get_answer(question: str, knowledge_base: dict) -> str | None:
    
    # Iterate over the questions in the knowledge base
    for i in knowledge_base["questions"]:

        # If the current question matches the input question
        if i["question"] == question:

            # Return the answer to the current question
            return i["answer"]

Interacting with the knowledge base and bot

In [22]:
def chat_bot():
    knowledge_base: dict = load_knowledge_base('knowledge_base.json')
    while True:
        # Get user input
        user_input: str = input('You: ')

        if user_input.lower() == 'quit':
            break

        # Answer the user's question using the knowledge base
        answer = answer_question(user_input, knowledge_base)

        # If the bot knows the answer, provide it
        if answer:
            # Add the user's question and answer to the knowledge base
            knowledge_base["questions"].append({"question": user_input, "answer": answer})
            save_knowledge_base('knowledge_base.json', knowledge_base)
            print(f'Bot: {answer}')
        else:
            # If the bot doesn't know the answer, ask the user to teach it
            print('Bot: I don\'t know the answer. Can you teach me?')
            new_answer: str = input('Type the answer or "skip" to skip: ')

            if new_answer.lower() != 'skip':
                # Add the user's question and answer to the knowledge base
                knowledge_base["questions"].append({"question": user_input, "answer": new_answer})
                save_knowledge_base('knowledge_base.json', knowledge_base)
                print('Bot: Thank you! I learned this new response')

We can also manually add questions to the bot so that it can have a larger knowledge base or have a function add them for us from sources such as our three books.

In [ ]:
# Run the chat bot
chat_bot()

Running the GUI

In [ ]:
!streamlit run gui.py